In [1]:
from greedy_mcmc_attack import *
from collections import defaultdict

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Pubmed"
pubmed_dataset = Planetoid(root='', name='Pubmed')
data = pubmed_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
model = GCN(data.x.shape[1], pubmed_dataset.num_classes, [16]).to(device)

In [5]:
model_save_path = "../models/pubmed_gcn_model.pth"
list_save_path = "../attacks/pubmed_gcn_edges.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [6]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [7]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [8]:
amts = defaultdict(int)

for _ in range(1):
    attacker = Metattack(data, device=device)
    attacker.setup_surrogate(
        model,
        labeled_nodes=data.train_mask,
        unlabeled_nodes=data.test_mask,
        lambda_=0.0,
    )
    attacker.reset()
    attacker.attack(2)

    for edge, itr in attacker._added_edges.items():
        amts[edge] = itr

    for edge, itr in attacker._removed_edges.items():
        amts[edge] = itr

sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=False)
sorted_keys = [key for key, value in sorted_list]

sorted_keys

Peturbing graph...:   0%|          | 0/10556 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6054341197013855
Surrogate acc on unlabled data: 0.798
Attack loss: 0.2960328161716461
Surrogate loss on unlabled data: 0.6533457040786743
Surrogate acc on unlabled data: 0.79
Attack loss: 0.3979365825653076
Surrogate loss on unlabled data: 0.7579464316368103
Surrogate acc on unlabled data: 0.78
Attack loss: 0.5146974325180054
Surrogate loss on unlabled data: 0.8564143180847168
Surrogate acc on unlabled data: 0.735
Attack loss: 0.6369403004646301
Surrogate loss on unlabled data: 0.9622113108634949
Surrogate acc on unlabled data: 0.718
Attack loss: 0.7189580202102661
Surrogate loss on unlabled data: 1.0664247274398804
Surrogate acc on unlabled data: 0.687
Attack loss: 0.8143057227134705
Surrogate loss on unlabled data: 1.210386037826538
Surrogate acc on unlabled data: 0.655
Attack loss: 0.9780105352401733
Surrogate loss on unlabled data: 1.1695771217346191
Surrogate acc on unlabled data: 0.64
Attack loss: 0.949097752571106
Surrogate loss on unlabled da

[(tensor(115, device='cuda:0'), tensor(2644, device='cuda:0')),
 (tensor(57, device='cuda:0'), tensor(2602, device='cuda:0')),
 (tensor(13, device='cuda:0'), tensor(2603, device='cuda:0')),
 (tensor(49, device='cuda:0'), tensor(1938, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(2513, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(1582, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(2697, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(1446, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(2633, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(2568, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(1005, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(2142, device='cuda:0')),
 (tensor(54, device='cuda:0'), tensor(2529, device='cuda:0')),
 (tensor(23, device='cuda:0'), tensor(2258, device='cuda:0')),
 (tensor(23, device='cuda:0'), tensor(422, device='cuda:0')),
 (tensor(23, device='cuda:0'), tensor(2149, device='cud

In [9]:
len(attacker._removed_edges)

525

In [10]:
len(sorted_list)

10556

In [11]:
new_edges = [(x.item(), y.item()) for (x, y) in sorted_keys]

In [26]:
len(new_edges)

10556

In [27]:
list_save_path = "../attacks/pubmed_gcn_edges_full_run.pth"
torch.save(new_edges, list_save_path)

In [13]:
# # save model
# model_save_path = "multi_metattack_gcn_model.pth"
# torch.save(model.state_dict(), model_save_path)

# # save edges
# list_save_path = "multi_metattack_edges_list.pth"
# torch.save(edges_to_add, list_save_path)

In [14]:
# loaded_model_state_dict = torch.load(model_save_path)

# model = GCN(data.x.shape[1], pubmed_dataset.num_classes, [16]).to(device)
# model.load_state_dict(loaded_model_state_dict)

# # Load the saved list
# edges_to_add = torch.load(list_save_path)

In [15]:
# model_save_path = "multi_metattack_gcn_model.pth"
# list_save_path = "multi_metattack_edges_list.pth"

In [16]:
# model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, data.x.shape[1], pubmed_dataset.num_classes, [16], device)

In [17]:
# G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.15)

In [18]:
# len(edges_to_add)

In [19]:
# def two_phase_attack(split):
#     diff_threshold = abs(initial_loss/200)
#     first_phase_edges = int(budget * split)
#     second_phase_percent = ptb_rate * (1 - split) * 1/2
#     print(second_phase_percent)
#     accuracies = []
#     G = to_networkx(data, to_undirected=True)
    
#     data_copy = copy.copy(data)
#     i, j = 0, 0 # i - number added, j - spot in list
#     while i < first_phase_edges:
#         u, v = edges_to_add[j]
    
#         G.add_edge(u, v)
    
#         modified_data = from_networkx(G).to(device)
#         modified_data.x = data.x 
#         modified_data.y = data.y 
#         modified_data.train_mask = data.train_mask
#         modified_data.test_mask = data.test_mask
    
#         modified_loss, modified_accuracy = train.test(modified_data)
#         # print(modified_loss)
    
#         if (abs(modified_loss - initial_loss) / max(modified_loss, initial_loss)) <= diff_threshold:
#         # if modified_accuracy == initial_accuracy:
#             # print(modified_accuracy, i)
#             i += 1
#             # accuracies.append(modified_accuracy)
#             accuracies.append(modified_loss)
#         else:
#             # print(i, 'miss!')
#             G.remove_edge(u, v)
            
#         j += 1
    
#     modified_data = from_networkx(G).to(device)
#     modified_data.x = data.x 
#     modified_data.y = data.y 
#     modified_data.train_mask = data.train_mask
#     modified_data.test_mask = data.test_mask
    
#     attacker = Metattack(modified_data, device=device)
#     attacker.setup_surrogate(model,
#                              labeled_nodes=data.train_mask,
#                              unlabeled_nodes=data.test_mask, lambda_=0.)
#     attacker.reset()
#     attacker.attack(second_phase_percent)

#     degs = defaultdict(tuple)
    
#     for k, v in attacker._added_edges.items():
#         degs[v] = (k, True)
        
#     for k, v in attacker._removed_edges.items():
#         degs[v] = (k, False)
    
#     for _, second in degs.items():
#         u, v = second[0]
#         if second[1]:
#             G.add_edge(u, v)
#         else:
#             G.remove_edge(u, v)
    
#         modified_data = from_networkx(G).to(device)
#         modified_data.x = data.x 
#         modified_data.y = data.y 
#         modified_data.train_mask = data.train_mask
#         modified_data.test_mask = data.test_mask
    
#         modified_loss, modified_accuracy = train.test(modified_data)
    
#         # accuracies.append(modified_accuracy)
#         accuracies.append(modified_loss)

#     print(accuracies)
#     return accuracies

In [20]:
# splits = [0, 0.5, 0.7, 0.9]
# split_dic_acc = defaultdict(list)
# split_dic_loss = defaultdict(list)
# itrs = defaultdict(int)

In [21]:
# for s in splits:
#     print(s)
#     split_dic_acc[s], split_dic_loss[s], itrs[s] = two_phase_attack_greedy(data, train, model, s, edges_to_add, constant_fn, device, verbose=True)

In [22]:
# plot_results(split_dic_acc, ptb_rate, "Greedy", "no", "constant", "accuracy")

In [23]:
# plot_results(split_dic_loss, ptb_rate, "Greedy", "no", "constant", "loss")

In [24]:
# itrs